#### CIE4604 Simulation and Visualization
# Module 2 Satellite Orbits - Exercise 5

**Hans van der Marel, 21 November 2020**

In this exercise we compute the time of a satellite overpass. A satellite overpass time is defined as the time the satellite is at is closest to an observer during its pass. 

## Import crsutil.py and tleplot.py Python modules

For the first time, download `CIE4604-M2-python.zip` from Brightspace and unzip this in your current working directory. This should give you two Python modules: `crsutil.py` and `tleplot.py`.

For this Jupyter notebook to work, the two Python modules should be in the same folder as this notebook. Import the module, and some other things we need, using the following statements:

In [ ]:
import numpy as np
import crsutil as crs
import tleplot as tle
from scipy.interpolate import interp1d

To get the docs for a specific function, use `help(functionname)`, or, to get the docs on all the functions at once do `help(module)`. To get help in a separate window type `module?` or `functionname?`. 

## Set user position

The position of the observer (latitude, longitude and height) is given in ECEF using the array `objcrd` in degrees/meters.
We convert this straightaway to Cartesian coordinates in an Earth Centered Earth Fixed (ECEF) frame.


In [ ]:
# Specify station position (latitude, longitude, height) 

objcrd=[ 52, 4.8,  0 ]

# Convert input to spherical coordinates in radians/meters

Re = 6378136                     # [m]   radius of the Earth 

lat = objcrd[0] * np.pi/180      # convert latitude from degrees to radians
lon = objcrd[1] * np.pi/180      # convert longitude from degrees to radians
Rs = Re + objcrd[2]              # convert height to radius (from CoM) using mean Earth radius Re 

# Position of the observer in ECEF (assume latitude and longitude are for spherical Earth)

xobje = Rs * np.array([[ np.cos(lat) * np.cos(lon), np.cos(lat) * np.sin(lon), np.sin(lat)]])
vobje = np.array([[ 0, 0, 0]])

## Compute satellite positions, lookangles and range-rate around time of overpass

This computation should be quite familiar by now (if you done the previous exercises). To compute the look-angles and range-rate we have to convert all coordinates to the same system! We have chosen to do all our computations in ECEF.

For the satellite positions we compute a period of 20 minutes for which we know that the satellite is near Delft (our station). Later we see how we can deal with this assumption and generalize the approach. 

In [ ]:
# Read TLE from file (we use an existing file, so that we have prior knowledge about the dates)
tleERS = tle.tleread('resource-10-oct-2017.tle', verbose=0)

# Compute satellite positions in ECI
t = tle.tledatenum(['2017-09-28 05:50:00',20,1])
xsat, vsat = tle.tle2vec(tleERS, t, 'RADARSAT-2')

# Convert ECI coordinate to ECEF
xsate, vsate = crs.eci2ecef(t, xsat, vsat)

# Compute lookangles and range-rate (range-rate is part of "lookangles")
lookangles, flags = crs.satlookanglesp(t, np.hstack([xsate, vsate]), xobje, verbose=1)

With the verbose option `satlookanglesp(...,verbose=1)` prints a nice table through a call to `prtlookangle`. 

Have a look at the range and range-rate that are printed. Around what time do you think the satellite is closest to the observer in Delft? What is the range-rate at this time?

How many minutes is the satellite visible?

## Find overpass time

The satellite is at is closest to an observer when the range-rate is zero. In the previous section we computed a table with the lookangles, range and range-rate (8'th column of lookangles, with index 7). 

Then we find the time the range-rate is zero by interpolating the range-rates, for this we use the `scipy` function `interp1d` (which is the Matlab equivalent of interp1)

In [ ]:
rangerate = lookangles[:,7]
set_interp = interp1d(rangerate, t, kind='linear')   # Matlab statement t0=interp1(rangerate,t,0,'pchip');
t0 = set_interp(0)

print('t0',crs.num2datetime(t0).isoformat())

t0 = np.array([t0])                                  # convert t0 to numpy array

Recompute the position, velocity and lookangles at t0

In [ ]:
xsat0, vsat0 = tle.tle2vec(tleERS, t0, 'RADARSAT-2')
xsate0, vsate0 = crs.eci2ecef(t0, xsat0, vsat0)
lookangles0, flags0 = crs.satlookanglesp(t0, np.hstack([xsate0, vsate0]), xobje, verbose=1)

Are you happy with the result? 

Please note that the interpolation is quite unusual in the sense that the range-rate is the independent variable and we seek the time t, while often this is the other way round...

We can improve the accuracy by reducing the interval to e.g. 6 seconds, as is shown below


In [ ]:
# Compute satellite positions in ECI using a 6 second interval
t = tle.tledatenum(['2017-09-28 05:50:00',20,0.1])
xsat, vsat = tle.tle2vec(tleERS, t, 'RADARSAT-2')

# Convert ECI coordinate to ECEF
xsate, vsate = crs.eci2ecef(t, xsat, vsat)

# Compute lookangles and range-rate (range-rate is part of "lookangles")
lookangles, flags = crs.satlookanglesp(t, np.hstack([xsate, vsate]), xobje, verbose=0)

# Interpolate within range-rate to find the time for zero range-rate
rangerate = lookangles[:,7]
set_interp = interp1d(rangerate, t, kind='linear')
t0 = np.array([set_interp(0)])

# Recompute the position, velocity and lookangles at t0
xsat0, vsat0 = tle.tle2vec(tleERS, t0, 'RADARSAT-2')
xsate0, vsate0 = crs.eci2ecef(t0, xsat0, vsat0)
lookangles0, flags0 = crs.satlookanglesp(t0, np.hstack([xsate0, vsate0]), xobje, verbose=1)

Happy now?

Observe that the lookangle at the satellite, with respect to the flight direction (LookAngle FlightDir), towards our station is very close to 90 degrees (right looking). This is the case in a ECEF reference frame.

In an ECI reference frame, as we have seen in other exercises, the heading of the satellite is not the same as in the ECEF reference frame. This is a consequence of the ECEF being a rotating frame, this affects the velocity vector, on which the heading depends. This also means that the FlighDir lookangle in an ECI reference frame and ECEF frame are not the same.

We can illustrate this by recomputing the lookangles in an ECI reference frame...

In [ ]:
# Convert station position from ECEF to ECI
xobj0, vobj0 = crs.ecef2eci(t0, xobje, vobje) 
# Compute look-angles in ECI
lookangles0b, flags0 = crs.satlookanglesp(t0, np.hstack([xsat0, vsat0]), np.hstack([xobj0, vobj0]), verbose=1)

There is about a 2.5 degree difference between the two lookangles (LookAngle FlightDir) and the two headings! The other quantities are the same.

## Putting it together - without cheating

In the previous sections we selected 20 minutes of data around a known
overpass, which is a bit of cheating... However this is easy to overcome.

The full algorithm would consist of the following steps
 
1. Find from the table of satellite positions the time of all Doppler sign transitions, 
2. Select only times which meet the swath definition with a 10 degree margin, 
3. Compute zero-Doppler time using the outcome of the previous step as start point, using the procedure outlined in the previous sections.

For the much needed variation, we will use SENTINEL-1A (instead of Radarsat) this time.

SENTINEL-1A has a 12 day repeat, so first generate 12 days of data @ 5 sec interval ...

In [ ]:
# Compute SENTINEL-1A satellite positions in ECI using a 5 second interval for a 12 day period, and convert to ECEF
t = tle.tledatenum(['2017-10-10 00:00', 12*24*60, 1/12]) 
xsat, vsat = tle.tle2vec(tleERS, t, 'SENTINEL-1A')
xsate, vsate = crs.eci2ecef(t, xsat, vsat)

#### 1. Find from the table of satellite position the time of all Doppler sign transitions

In [ ]:
# Compute lookangles and range-rate (range-rate is part of "lookangles")
lookangles, flags = crs.satlookanglesp(t, np.hstack([xsate, vsate]), xobje, verbose=0)
rangerate = lookangles[:,7]

# Find all range rate (Doppler) sign transitions 
ipositive = rangerate > 0
itransit = np.logical_xor(ipositive[0:-1], ipositive[1:])
itransit =  np.concatenate(( [False], itransit))

#### 2. Select only candidates which meet the swath definition with a marging of 10 degrees

In [ ]:
# Select only incidence angles below 50 degrees
itransit = np.logical_and( itransit, lookangles[:,0] < 50*np.pi/180 )
# Print the candidates
lookangles0, flags0 = crs.satlookanglesp(t[itransit], np.hstack([xsate[itransit,:], vsate[itransit,:]]), xobje, verbose=1)
tcandidate=t[itransit]

#### 3. Compute zero-Doppler time for each candidate

In [ ]:
# refine the overpass times (using interpolation)
t0=tcandidate
idxtransit = np.nonzero(itransit)[0]
for l in range(idxtransit.size):
    k = idxtransit[l]
    k1 = max(k-5,0)
    k2 = min(k+5,itransit.size-1)
    # Interpolate within range-rate to find the time for zero range-rate
    set_interp = interp1d(rangerate[k1:k2], t[k1:k2], kind='linear')
    t0[l] = np.array([set_interp(0)])

# Recompute the position, velocity and lookangles at t0
xsat0, vsat0 = tle.tle2vec(tleERS, tcandidate, 'SENTINEL-1A')
xsate0, vsate0 = crs.eci2ecef(t0, xsat0, vsat0)
lookangles0, flags0 = crs.satlookanglesp(t0, np.hstack([xsate0, vsate0]), xobje, verbose=1)

In the real world we would add another test to single out only those Sentinel-1A passes that are within the satellite swath. This means for SENTINEL-1A

- right-looking (RL), and,
- incidence angle between *29.16* and *46.0* degrees.

This is done with the following code using the previous computed lookangles

In [ ]:
inswath = (lookangles0[:,0] < 46*np.pi/180) & (lookangles0[:,0] > 29.16*np.pi/180) & (lookangles0[:,4] < np.pi)
lookanglesInSwath, flagsInSwath = crs.satlookanglesp(t0[inswath], np.hstack([xsate0[inswath,:], vsate0[inswath,:]]), 
                                                     xobje, verbose=1)

This is our final answer!

So for Sentinel-1A, Delft is observed four times during a repeat period of 12 days. On two ascending tracks, and on two descending tracks.

[End of this Jupyter notebook]